In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')
# model.eval()

Some weights of the model checkpoint at /home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
query = '宿泊代金の支払方法について教えてください。'

In [8]:
tokenized_text = ['宿泊代金', '支払方法', '教える']

In [4]:
tokenized_text = tokenizer.tokenize(query)
tokenized_text

['宿泊', '代金', 'の', '支払', '方法', 'について', '教え', 'て', 'ください', '。']

In [5]:
# ['テレビ', 'で', 'サッカー', 'の', '試合', 'を', '見る', '。']

# Mask a token that we will try to predict back with `BertForMaskedLM`
# masked_index = question.index(word)
masked_index = 1
# tokenized_text[masked_index] = '[MASK]'
# ['テレビ', 'で', '[MASK]', 'の', '試合', 'を', '見る', '。']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# [571, 12, 4, 5, 608, 11, 2867, 8]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

In [6]:
outputs = model(tokens_tensor)

In [11]:
outputs[0][0]

tensor([[ -5.4410,   3.8274,  -3.9452,  ...,  -8.0093,  -5.5738,  -7.6920],
        [ -3.9263,   7.1168,  -5.4834,  ...,  -8.1102,  -5.3262,  -3.4483],
        [ -5.8970,   8.7722,  -2.9035,  ...,  -6.5889,  -3.9831,  -3.8655],
        ...,
        [ -8.7810,   3.7610,  -6.2852,  ..., -12.3828,  -6.7486, -11.1676],
        [ -8.9119,   3.8230,  -3.5011,  ..., -11.6964,  -3.7670, -12.4521],
        [ -5.8612,   3.1078,  -2.5902,  ..., -10.4407,  -5.2364, -11.5876]],
       grad_fn=<SelectBackward>)

In [14]:
# # Predict
# with torch.no_grad():
outputs = model(tokens_tensor)
predictions = outputs[0][0, masked_index].topk(10)  # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

0 教え
1 教える
2 学び
3 を
4 学ぶ
5 生きる
6 芸能
7 教育
8 学習
9 も


In [5]:
x = outputs[0][0, masked_index].topk(10)

NameError: name 'outputs' is not defined

In [7]:
for i, index_t in enumerate(x.indices):
    index = index_t.item()
    print(tokenizer.convert_ids_to_tokens([index]))index

SyntaxError: invalid syntax (<ipython-input-7-498270fda576>, line 3)

## Trying Next seq approach

In [3]:
import torch
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

In [4]:
# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')
# Load pre-trained model
model = BertForMaskedLM.from_pretrained('/home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new')


Some weights of the model checkpoint at /home/iftekhar/Downloads/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask/new were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
test = '朝食に[MASK]を焼いて食べました。'

In [6]:
token_ids = tokenizer.encode(test, add_special_tokens=True)

In [7]:
token_ids

[2, 25965, 7, 4, 11, 16878, 16, 2949, 3913, 10, 8, 3]

In [8]:
tokens = tokenizer.convert_ids_to_tokens(token_ids)

In [9]:
tokens

['[CLS]', '朝食', 'に', '[MASK]', 'を', '焼い', 'て', '食べ', 'まし', 'た', '。', '[SEP]']

In [10]:
token_ids = torch.tensor([token_ids])

In [12]:
token_ids

tensor([[    2, 25965,     7,     4,    11, 16878,    16,  2949,  3913,    10,
             8,     3]])

In [22]:
masked_index = torch.where(token_ids == tokenizer.mask_token_id)[1].tolist()[0]
print(masked_index)

3


In [25]:
result = model(token_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = token_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

[CLS] 朝食 に パン を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に 肉 を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に ご飯 を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に 豚肉 を 焼い て 食べ まし た 。 [SEP]
[CLS] 朝食 に ハム を 焼い て 食べ まし た 。 [SEP]


In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# Tokenize input
text = 'テレビでサッカーの試合を見る。'
tokenized_text = tokenizer.tokenize(text)
# ['テレビ', 'で', 'サッカー', 'の', '試合', 'を', '見る', '。']

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 2
tokenized_text[masked_index] = '[MASK]'
# ['テレビ', 'で', '[MASK]', 'の', '試合', 'を', '見る', '。']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# [571, 12, 4, 5, 608, 11, 2867, 8]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

# Load pre-trained model
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(5) # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 クリケット
1 タイガース
2 サッカー
3 メッツ
4 カブス


In [3]:
model.save_pretrained('/home/iftekhar/kuhuto_cl_bert/')

In [5]:
tokenizer.save_pretrained('/home/iftekhar/kuhuto_cl_bert/')

('/home/iftekhar/kuhuto_cl_bert/tokenizer_config.json',
 '/home/iftekhar/kuhuto_cl_bert/special_tokens_map.json',
 '/home/iftekhar/kuhuto_cl_bert/vocab.txt',
 '/home/iftekhar/kuhuto_cl_bert/added_tokens.json')